# 【使い方】

ランタイム>すべてのセルを実行（**Ctrl+F9**）によりすべてのセルを実行し、セル[2]の最後に生成された**URL（Running on public URL）をクリック**して開いてください。（GUIが新しいタブで開かれる）

※このcolabの画面（タブ）は閉じないでください。




# [How to use]

Run all cells by selecting Runtime > Run all cells (**Ctrl+F9**), and **click the URL generated at the end of Cell [2] (Running on public URL)** to open it. (The GUI will open in a new tab)

Note: Please do not close this Colab screen (tab).

In [ ]:
!pip install gradio pdf2image
!apt-get install poppler-utils

In [ ]:
from google.colab import files

# **ColabでPDFと参照画像をアップロード**
print("PDFをアップロードしてください")
uploaded_pdf = files.upload()
pdf_file_path = list(uploaded_pdf.keys())[0]

print("参照画像（サイズ調整用）をアップロードしてください")
uploaded_ref_image = files.upload()
reference_image_path = list(uploaded_ref_image.keys())[0]

In [ ]:
import os
import zipfile
from PIL import Image
import numpy as np
from pdf2image import convert_from_path, pdfinfo_from_path
from scipy.ndimage import binary_dilation

import shutil

# カラーマスク用の色とカテゴリID
color_to_category_id = {
    (255, 0, 0, 255): 1,     # 赤
    (0, 0, 255, 255): 2,     # 青
    (0, 255, 0, 255): 3,     # 緑
    (255, 255, 0, 255): 4,   # 黄
    (128, 0, 128, 255): 5,   # 紫
    (255, 165, 0, 255): 6,   # オレンジ
    (0, 255, 255, 255): 7,   # 水色
    (173, 255, 47, 255): 8,  # 黄緑
    (128, 128, 128, 255): 9, # グレー
    (0, 128, 128, 255): 10,  # 青緑
    (255, 192, 203, 255): 11,# ピンク
    (255, 20, 147, 255): 12, # ローズ
    (0, 128, 0, 255): 13,    # オリーブ
    (128, 0, 0, 255): 14,    # マルーン
    (0, 255, 230, 255): 15,  # シアン
    (255, 215, 0, 255): 16,  # ゴールド
    (255, 69, 0, 255): 17,   # インディアンレッド
    (0, 0, 128, 255): 18,    # ネイビーブルー
    (220, 20, 60, 255): 19,  # クリムゾンレッド
    (128, 128, 0, 255): 20   # オリーブグリーン
}

# **画像フォルダをリセット**
def reset_folder(folder_path):
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)  # 古いファイルを削除
    os.makedirs(folder_path)  # 新しくフォルダを作成

# **/content フォルダ内の最初のPDFファイルを取得**
def get_first_pdf():
    pdf_files = [f for f in os.listdir('/content') if f.lower().endswith('.pdf')]
    if not pdf_files:
        raise FileNotFoundError("PDFファイルが /content に見つかりません！")
    return os.path.join('/content', pdf_files[0])

# **/content フォルダ内の最初の画像ファイル（PNG/JPG）を取得**
def get_first_image():
    image_files = [f for f in os.listdir('/content') if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    if not image_files:
        raise FileNotFoundError("画像ファイル（PNG/JPG）が /content に見つかりません！")
    return os.path.join('/content', image_files[0])

# **PDFをPNGに変換する関数（image_paths をリセット）**
def convert_pdf_to_png(pdf_path, start_page, end_page):
    output_folder = '/content/images'
    reset_folder(output_folder)  # **フォルダをリセット**

    image_paths = []  # **リストを明示的にリセット**

    for i, page in enumerate(range(start_page, end_page + 1)):
        images = convert_from_path(pdf_path, first_page=page, last_page=page)  # 1ページずつ処理
        if images:
            image_path = os.path.join(output_folder, f'image{page:04}.png')  # 開始ページ番号を付ける
            images[0].save(image_path, 'PNG')
            image_paths.append(image_path)
            del images  # メモリ解放

    return image_paths  # **新しい `image_paths` を返す**

# **画像を参照画像に合わせてリサイズ**
def resize_images(image_paths, reference_image_path):
    ref_img = Image.open(reference_image_path)
    width, height = ref_img.size

    for image_path in image_paths:
        with Image.open(image_path) as img:
            resized_img = img.resize((width, height))
            resized_img.save(image_path)

    return f"リサイズが完了しました: {len(image_paths)} 枚"

# **カラー領域を抽出し、バイナリ画像を生成（image_paths を明示的に受け取る）**
def extract_and_save_colors(image_paths, dilation_pixels, num_categories, start_page):
    zip_folder_path = '/content/extracted_colors'
    reset_folder(zip_folder_path)  # **フォルダをリセット**

    structure = np.ones((2 * dilation_pixels + 1, 2 * dilation_pixels + 1), dtype=bool)

    # 指定されたカテゴリ数まで処理
    limited_color_to_category_id = dict(list(color_to_category_id.items())[:num_categories])

    for image_index, image_path in enumerate(image_paths):  # **image_paths は最新のものを受け取る**
        image = Image.open(image_path).convert('RGBA')
        data = np.array(image)

        for color, category_id in limited_color_to_category_id.items():
            color_areas = (data[:, :, 0] == color[0]) & (data[:, :, 1] == color[1]) & (data[:, :, 2] == color[2])
            binary_image = np.zeros((data.shape[0], data.shape[1]), dtype=np.uint8)
            binary_image[color_areas] = 255

            binary_image = binary_dilation(binary_image, structure=structure).astype(np.uint8) * 255

            label_dir = os.path.join(zip_folder_path, f'label{category_id:02}')
            os.makedirs(label_dir, exist_ok=True)
            file_name = f'image_{start_page + image_index:03d}.png'  # **開始ページからの連番**
            Image.fromarray(binary_image).save(os.path.join(label_dir, file_name))

    zip_path = '/content/extracted_colors.zip'
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for root, dirs, files in os.walk(zip_folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, zip_folder_path))

    return zip_path

# **メイン処理**
def process_and_download(pdf_path, reference_image_path, start_page, end_page, dilation_pixels, num_categories):
    image_paths = convert_pdf_to_png(pdf_path, start_page, end_page)  # **最新の `image_paths` を取得**
    resize_images(image_paths, reference_image_path)  # **リサイズ**
    zip_path = extract_and_save_colors(image_paths, dilation_pixels, num_categories, start_page)  # **image_paths を渡す**

    # 自動ダウンロード
    files.download(zip_path)

# **PDFファイルと参照画像を取得**
pdf_file_path = get_first_pdf()
reference_image_path = get_first_image()

print(f"使用するPDFファイル: {pdf_file_path}")
print(f"使用する参照画像: {reference_image_path}")


from pdf2image import pdfinfo_from_path  # 必要な関数をインポート

# **メモリ負荷をかけずにPDFの総ページ数を取得**
pdf_info = pdfinfo_from_path(pdf_file_path)  # PDFの情報を取得
total_pages = pdf_info["Pages"]  # ページ数を取得
print(f"このPDFの総ページ数: {total_pages} ページ")


# **パラメータを入力**
start_page = int(input("開始ページを入力: "))
end_page = int(input(f"終了ページを入力（全{total_pages}ページ）: "))
dilation_pixels = int(input("拡大ピクセル数: "))
num_categories = int(input("処理するカテゴリ数（最大20）: "))

# **処理を実行**
process_and_download(pdf_file_path, reference_image_path, start_page, end_page, dilation_pixels, num_categories)


Copyright (c) 2024 Satoru Muro. All rights reserved.
Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:
1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.